## 機器學習百日馬拉松期中考 - Enron Fraud Dataset 安隆公司詐欺案資料集
安隆公司曾是一間能源公司，2001 年破產前是世界上最大的電力、天然氣及電信公司之一。擁有上千億資產的公司於 2002 年竟然在短短幾周內宣告破產，才揭露其財報在多年以來均是造假的醜聞。在本資料集中你將會扮演偵探的角色，透過高層經理人內部的 mail 來往的情報以及薪資、股票等財務特徵，訓練出一個機器學習模型來幫忙你找到可疑的詐欺犯罪者是誰! 我們已經先幫你找到幾位犯罪者 (Person-of-Interest, poi) 與清白的員工，請利用這些訓練資料來訓練屬於自己的詐欺犯機器學習模型吧!

## 特徵說明
有關財務的特徵: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (單位皆為美元)。更詳細的特徵說明請參考 enron61702insiderpay.pdf 的最後一頁(請至Data頁面參考該PDF檔)

有關 email 的特徵: ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (除了 email_address，其餘皆為次數)

嫌疑人的標記，也就是我們常用的 **y**。POI label: [‘poi’] (boolean, represented as integer)

我們也建議你對既有特徵進行一些特徵工程如 rescale, transform ，也試著發揮想像力與創意，建立一些可以幫助找到嫌疑犯的特徵，增進模型的預測能力，

## 關鍵問題
如果你是第一次實作機器學習專案，一開始可能會有些迷惘，不曉得該從何著手，我們提供了一系列的問題，這些都是一個機器學習專案中必須要回答的問題，可以試著從這些問題開始！

## 專案結束後你可以學會
如何處理存在各種缺陷的真實資料
使用 val/test data 來了解機器學習模型的訓練情形
使用適當的評估函數了解預測結果
應用適當的特徵工程提升模型的準確率
調整機器學習模型的超參數來提升準確率
清楚的說明文件讓別人了解你的成果

In [1]:
# 載入套件與資料
import pandas as pd
import numpy as np
from sklearn import datasets, metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

### 載入資料

In [2]:
data_path = 'data/Midterm/'
train_data = pd.read_csv(data_path + 'train_data.csv')
test_data  = pd.read_csv(data_path + 'test_features.csv')
print("train_data.shape: ", train_data.shape)
print("test_data.shape : ", test_data.shape)

train_data.shape:  (113, 22)
test_data.shape :  (33, 21)


In [3]:
train_data.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


### 檢查欄位缺值數量 (去掉.head()可以顯示全部)

In [4]:
# 訓練集資料缺失值
train_data.isnull().sum().sort_values(ascending=False).head()

loan_advances                111
restricted_stock_deferred    103
director_fees                100
deferral_payments             85
deferred_income               79
dtype: int64

In [5]:
# 測試集資料缺失值
test_data.isnull().sum().sort_values(ascending=False).head()

loan_advances                31
director_fees                29
restricted_stock_deferred    25
deferral_payments            22
deferred_income              18
dtype: int64

### 觀察資料類型

In [6]:
dtype_df = train_data.dtypes.reset_index()
dtype_df

,index,0
0,name,object
1,bonus,float64
2,deferral_payments,float64
3,deferred_income,float64
4,director_fees,float64
5,email_address,object
6,exercised_stock_options,float64
7,expenses,float64
8,from_messages,float64
9,from_poi_to_this_person,float64


In [7]:
# 秀出資料欄位的類型與數量
# 重新定義欄位名稱
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,bool,1
1,float64,19
2,object,2


## 準備訓練模型所需資料

In [8]:
df_train = train_data.drop(['poi'] , axis=1)
df_test  = test_data

In [9]:
df_train["deferred_income"].isnull().sum(axis = 0)

79

### 處理資料

In [10]:
object_type = list([])
float_type = list([])

# Iterate through the columns
for idx, col in enumerate(df_train):
    null_count = df_train[col].isna().sum()
    print("df_train[{0}] column: {1}, isnull_count:{2} ".format(idx, col, null_count))
    #print("df_train[{0}] column: {1}".format(idx, col))
    if df_train[col].dtype == 'object':
        object_type.append(col)
    if df_train[col].dtype == 'float64':
        float_type.append(col)

print("object_type: ", object_type)
print("float_type : ", float_type)

df_train[0] column: name, isnull_count:0 
df_train[1] column: bonus, isnull_count:52 
df_train[2] column: deferral_payments, isnull_count:85 
df_train[3] column: deferred_income, isnull_count:79 
df_train[4] column: director_fees, isnull_count:100 
df_train[5] column: email_address, isnull_count:30 
df_train[6] column: exercised_stock_options, isnull_count:32 
df_train[7] column: expenses, isnull_count:40 
df_train[8] column: from_messages, isnull_count:48 
df_train[9] column: from_poi_to_this_person, isnull_count:48 
df_train[10] column: from_this_person_to_poi, isnull_count:48 
df_train[11] column: loan_advances, isnull_count:111 
df_train[12] column: long_term_incentive, isnull_count:64 
df_train[13] column: other, isnull_count:44 
df_train[14] column: restricted_stock, isnull_count:31 
df_train[15] column: restricted_stock_deferred, isnull_count:103 
df_train[16] column: salary, isnull_count:40 
df_train[17] column: shared_receipt_with_poi, isnull_count:48 
df_train[18] column: to_

In [11]:
df_train.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,6.100000e+01,2.800000e+01,3.400000e+01,13.000000,8.100000e+01,73.000000,65.000000,65.000000,65.000000,2.000000e+00,4.900000e+01,6.900000e+01,8.200000e+01,10.000000,7.300000e+01,65.000000,65.000000,9.600000e+01,9.800000e+01
mean,1.147436e+06,6.344374e+05,-4.625664e+05,89397.846154,2.985081e+06,51040.547945,711.323077,64.800000,40.092308,4.096250e+07,7.926171e+05,4.471774e+05,1.294855e+06,-221885.700000,2.739025e+05,1111.369231,2156.061538,2.590977e+06,3.527136e+06
std,1.505189e+06,8.603646e+05,8.095392e+05,41143.391399,6.004174e+06,47596.682104,2074.497628,91.863214,88.901407,5.736404e+07,9.504645e+05,1.341564e+06,2.498335e+06,205191.374121,1.716647e+05,1165.852016,2811.676718,1.056645e+07,7.182997e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,7.102300e+04,2.000000e+00,4.409300e+04,-560222.000000,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.500000e+05,7.656750e+04,-5.527032e+05,101250.000000,4.004780e+05,18834.000000,19.000000,10.000000,0.000000,2.068125e+07,2.750000e+05,9.720000e+02,2.689220e+05,-389621.750000,2.061210e+05,178.000000,517.000000,3.024025e+05,4.211518e+05
50%,7.500000e+05,1.951900e+05,-1.175340e+05,108579.000000,8.500100e+05,41953.000000,45.000000,28.000000,7.000000,4.096250e+07,4.221580e+05,5.238200e+04,4.628225e+05,-139856.500000,2.516540e+05,599.000000,1088.000000,1.106740e+06,9.979710e+05
75%,1.000000e+06,8.342052e+05,-2.708325e+04,112492.000000,2.165172e+06,59175.000000,215.000000,88.000000,27.000000,6.124375e+07,8.318090e+05,3.620960e+05,9.664905e+05,-77953.250000,2.885890e+05,1902.000000,2649.000000,1.985668e+06,2.493616e+06
max,8.000000e+06,2.964506e+06,-1.042000e+03,125034.000000,3.434838e+07,228763.000000,14368.000000,528.000000,411.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,44093.000000,1.111258e+06,4527.000000,15149.000000,1.035598e+08,4.911008e+07


In [12]:
# object type 丟棄
df_train = df_train.drop(['name', 'email_address'] , axis=1)
df_test  = df_test.drop(['name', 'email_address'] , axis=1)

# float type 缺失值補
for idx, col in enumerate(float_type):
    #print("X_train[{0}]".format(col))
    # fill "NaN" string as 
    #df_train.loc[(df_train[col].isnull()), [col]] = df_train[col].mean()
    df_train.loc[(df_train[col].isnull()), [col]] = df_train[col].mean()
    #df_test.loc[(df_test[col].isnull()), [col]] = df_test[col].mean()
    df_test.loc[(df_test[col].isnull()), [col]] = df_test[col].mean()

In [13]:
df_train.head(10)

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1.750000e+06,634437.428571,-3.504386e+06,89397.846154,1.979418e+07,46950.0,18.000000,42.0,4.000000,40962500.0,1.617011e+06,174839.0,2.748364e+06,-221885.7,4.206360e+05,864.000000,905.000000,505050.0,22542539.0
1,5.600000e+06,634437.428571,-4.625664e+05,89397.846154,1.925000e+07,29336.0,108.000000,88.0,30.000000,40962500.0,1.920000e+06,22122.0,6.843672e+06,-221885.7,1.111258e+06,2042.000000,3627.000000,8682716.0,26093672.0
2,2.000000e+05,634437.428571,-4.167000e+03,89397.846154,1.624396e+06,22884.0,39.000000,13.0,14.000000,40962500.0,7.926171e+05,1573324.0,8.692200e+05,-221885.7,2.118440e+05,91.000000,225.000000,2003885.0,2493616.0
3,8.000000e+05,634437.428571,-4.625664e+05,89397.846154,2.985081e+06,118134.0,711.323077,64.8,40.092308,40962500.0,6.026710e+05,907502.0,9.850320e+05,-221885.7,2.243050e+05,1111.369231,2156.061538,2652612.0,985032.0
4,1.250000e+06,634437.428571,-2.625000e+05,89397.846154,2.985081e+06,35818.0,144.000000,199.0,25.000000,40962500.0,3.753040e+05,486.0,1.260270e+05,-221885.7,2.401890e+05,2188.000000,2598.000000,1639297.0,126027.0
5,1.147436e+06,10259.000000,-4.625664e+05,89397.846154,3.076606e+07,77978.0,711.323077,64.8,40.092308,40962500.0,7.926171e+05,2856.0,1.294855e+06,-221885.7,2.739025e+05,1111.369231,2156.061538,91093.0,30766064.0
6,1.147436e+06,634437.428571,-4.625664e+05,89397.846154,8.308552e+06,53947.0,711.323077,64.8,40.092308,40962500.0,7.926171e+05,147950.0,3.576206e+06,-221885.7,1.584030e+05,1111.369231,2156.061538,360300.0,11884758.0
7,1.200000e+06,27610.000000,-1.440620e+05,89397.846154,2.985081e+06,16514.0,40.000000,240.0,11.000000,40962500.0,7.926171e+05,101740.0,6.982420e+05,-221885.7,2.885420e+05,1132.000000,1758.000000,1490344.0,698242.0
8,7.000000e+06,202911.000000,-3.000000e+05,89397.846154,3.434838e+07,99832.0,36.000000,123.0,16.000000,81525000.0,3.600000e+06,10359729.0,1.476169e+07,-221885.7,1.072321e+06,2411.000000,4273.000000,103559793.0,49110078.0
9,6.000000e+05,634437.428571,-4.625664e+05,89397.846154,3.847280e+05,125978.0,16.000000,52.0,6.000000,40962500.0,7.102300e+04,200308.0,3.938180e+05,-221885.7,2.749750e+05,874.000000,873.000000,1272284.0,778546.0


In [14]:
# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

scaler.fit(df_train)
X_train = scaler.transform(df_train)
X_train

array([[2.11853720e-01, 2.40279096e-01, 0.00000000e+00, ...,
        5.61887092e-02, 4.87547056e-03, 4.59505908e-01],
       [6.97351828e-01, 2.40279096e-01, 8.68261766e-01, ...,
        2.36549165e-01, 8.38412299e-02, 5.31750703e-01],
       [1.63934426e-02, 2.40279096e-01, 9.99107995e-01, ...,
        1.11317254e-02, 1.93486275e-02, 5.16275414e-02],
       ...,
       [2.90037831e-02, 2.40279096e-01, 8.68261766e-01, ...,
        5.57248874e-02, 1.33888253e-02, 1.48290569e-03],
       [1.35868398e-01, 2.40279096e-01, 8.68261766e-01, ...,
        3.41902995e-02, 3.15760063e-06, 4.60061060e-02],
       [7.31399748e-02, 2.40279096e-01, 8.68261766e-01, ...,
        1.00000000e+00, 1.02105410e-02, 2.09767346e-02]])

In [15]:
scaler.fit(df_test)
X_test = scaler.transform(df_test)
X_test

array([[5.29597628e-02, 6.53298569e-02, 9.16633461e-01, 2.79399980e-01,
        6.01984424e-04, 2.90538189e-03, 1.54399738e-01, 9.42148760e-01,
        1.77339901e-01, 5.00000000e-01, 6.92352859e-02, 4.93651846e-03,
        2.07787988e-02, 3.57250076e-01, 5.16752586e-03, 1.00000000e+00,
        1.00000000e+00, 1.77468708e-02, 2.44762601e-03],
       [1.28543139e-02, 1.29799135e-01, 1.00000000e+00, 2.79399980e-01,
        5.40201807e-02, 1.21833418e-02, 4.90677134e-03, 5.78512397e-01,
        2.46305419e-02, 5.00000000e-01, 1.86794525e-02, 3.63742725e-05,
        2.14896111e-02, 3.57250076e-01, 7.59363418e-03, 2.88147880e-01,
        2.26900290e-01, 8.60784766e-03, 4.71274594e-04],
       [1.43968315e-02, 1.29799135e-01, 8.88676352e-01, 2.79399980e-01,
        1.53444066e-02, 6.09358181e-03, 6.54236179e-03, 1.32231405e-01,
        3.44827586e-02, 5.00000000e-01, 3.19443053e-02, 2.64393150e-04,
        2.60109646e-02, 3.57250076e-01, 6.26765305e-03, 1.87024284e-01,
        1.24416992e-01

In [16]:
# training set target
le = LabelEncoder()
le.fit(train_data['poi'])
df_target = le.transform(train_data['poi'])
df_target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

## 訓練模型

In [17]:
# 切分訓練集/測試集
X_train, X_test, y_train, y_test = train_test_split(df_train, df_target, test_size=0.25, random_state=5)

# # 建立模型
# clf = GradientBoostingClassifier()
# # 訓練模型
# clf.fit(X_train, y_train)
# # 預測測試集
# y_pred = clf.predict(X_test)
# acc = metrics.accuracy_score(y_test, y_pred)
# print("Acuuracy: ", acc)

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(X_train, y_train)

# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(df_test)[:, 1]
log_reg_pred

array([2.61150135e-07, 9.99923882e-01, 3.68742483e-06, 6.33989564e-06,
       9.99979984e-01, 9.98417355e-01, 1.00000000e+00, 1.00000000e+00,
       1.14679041e-04, 2.08530416e-01, 9.99999991e-01, 9.99999997e-01,
       1.00000000e+00, 9.99935440e-01, 7.23120475e-07, 1.00000000e+00,
       5.70287416e-02, 8.31273906e-04, 9.99999999e-01, 1.00000000e+00,
       1.00000000e+00, 5.41771631e-06, 9.99994506e-01, 9.99999022e-01,
       9.99926606e-01, 1.22531820e-01, 1.00000000e+00, 9.99999999e-01,
       2.29657921e-05, 7.98918848e-01, 1.00000000e+00, 9.92837268e-01,
       1.16020135e-04])

## 寫入submission.csv

In [18]:
submit_data = pd.read_csv(data_path + 'sample_submission.csv')
submit_data["poi"] = log_reg_pred
submit_data.to_csv(data_path + 'sample_submission.csv', index=False)
submit_data

,name,poi
0,BELDEN TIMOTHY N,2.611501e-07
1,BOWEN JR RAYMOND M,9.999239e-01
2,HANNON KEVIN P,3.687425e-06
3,DELAINEY DAVID W,6.339896e-06
4,CAUSEY RICHARD A,9.999800e-01
5,HICKERSON GARY J,9.984174e-01
6,FREVERT MARK A,1.000000e+00
7,CHAN RONNIE,1.000000e+00
8,DONAHUE JR JEFFREY M,1.146790e-04
9,REYNOLDS LAWRENCE,2.085304e-01
